<h1>Содержание<span class="tocSkip"></span></h1>
<div class="toc"><ul class="toc-item"><li><span><a href="#Подготовка-данных" data-toc-modified-id="Подготовка-данных-1"><span class="toc-item-num">1&nbsp;&nbsp;</span>Подготовка данных</a></span></li><li><span><a href="#Исследование-задачи" data-toc-modified-id="Исследование-задачи-2"><span class="toc-item-num">2&nbsp;&nbsp;</span>Исследование задачи</a></span></li><li><span><a href="#Борьба-с-дисбалансом" data-toc-modified-id="Борьба-с-дисбалансом-3"><span class="toc-item-num">3&nbsp;&nbsp;</span>Борьба с дисбалансом</a></span></li><li><span><a href="#Тестирование-модели" data-toc-modified-id="Тестирование-модели-4"><span class="toc-item-num">4&nbsp;&nbsp;</span>Тестирование модели</a></span></li></ul></div>

# Отток клиентов

Из «Бета-Банка» стали уходить клиенты. Каждый месяц. Немного, но заметно. Банковские маркетологи посчитали: сохранять текущих клиентов дешевле, чем привлекать новых.

Нужно спрогнозировать, уйдёт клиент из банка в ближайшее время или нет. Вам предоставлены исторические данные о поведении клиентов и расторжении договоров с банком. 

Постройте модель с предельно большим значением *F1*-меры. Чтобы сдать проект успешно, нужно довести метрику до 0.59. Проверьте *F1*-меру на тестовой выборке самостоятельно.

Дополнительно измеряйте *AUC-ROC*, сравнивайте её значение с *F1*-мерой.

Источник данных: [https://www.kaggle.com/barelydedicated/bank-customer-churn-modeling](https://www.kaggle.com/barelydedicated/bank-customer-churn-modeling)

## Подготовка данных

In [1]:
import pandas as pd # импортируем необходимые библиотеки
import numpy as np
from sklearn.preprocessing import StandardScaler 
from sklearn.model_selection import train_test_split
from sklearn.linear_model import LogisticRegression
from sklearn.tree import DecisionTreeClassifier 
from sklearn.ensemble import RandomForestClassifier 
from sklearn.metrics import f1_score
from sklearn.metrics import roc_auc_score, roc_curve
from sklearn.utils import shuffle
from sklearn.metrics import accuracy_score
from sklearn.dummy import DummyClassifier

In [2]:
df = pd.read_csv("/datasets/Churn.csv") # прочитаю и запишу датасет в переменную df

In [3]:
df.info() # просмотрю общую информацию о датасете

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 10000 entries, 0 to 9999
Data columns (total 14 columns):
 #   Column           Non-Null Count  Dtype  
---  ------           --------------  -----  
 0   RowNumber        10000 non-null  int64  
 1   CustomerId       10000 non-null  int64  
 2   Surname          10000 non-null  object 
 3   CreditScore      10000 non-null  int64  
 4   Geography        10000 non-null  object 
 5   Gender           10000 non-null  object 
 6   Age              10000 non-null  int64  
 7   Tenure           9091 non-null   float64
 8   Balance          10000 non-null  float64
 9   NumOfProducts    10000 non-null  int64  
 10  HasCrCard        10000 non-null  int64  
 11  IsActiveMember   10000 non-null  int64  
 12  EstimatedSalary  10000 non-null  float64
 13  Exited           10000 non-null  int64  
dtypes: float64(3), int64(8), object(3)
memory usage: 1.1+ MB


In [4]:
df.head() # выведу первые 5 строк датасета

,RowNumber,CustomerId,Surname,CreditScore,Geography,Gender,Age,Tenure,Balance,NumOfProducts,HasCrCard,IsActiveMember,EstimatedSalary,Exited
0,1,15634602,Hargrave,619,France,Female,42,2.0,0.00,1,1,1,101348.88,1
1,2,15647311,Hill,608,Spain,Female,41,1.0,83807.86,1,0,1,112542.58,0
2,3,15619304,Onio,502,France,Female,42,8.0,159660.80,3,1,0,113931.57,1
3,4,15701354,Boni,699,France,Female,39,1.0,0.00,2,0,0,93826.63,0
4,5,15737888,Mitchell,850,Spain,Female,43,2.0,125510.82,1,1,1,79084.10,0


In [5]:
df.duplicated().sum() # проверю дубликаты

0

In [6]:
df['Tenure']=df['Tenure'].fillna(0).astype('int64') # пропуски в столбце с тем, сколько лет клиент является абонентом заменю на 0

In [7]:
df.info() # ещё раз просмотрю общую информацию о датасете после обработки пропусков

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 10000 entries, 0 to 9999
Data columns (total 14 columns):
 #   Column           Non-Null Count  Dtype  
---  ------           --------------  -----  
 0   RowNumber        10000 non-null  int64  
 1   CustomerId       10000 non-null  int64  
 2   Surname          10000 non-null  object 
 3   CreditScore      10000 non-null  int64  
 4   Geography        10000 non-null  object 
 5   Gender           10000 non-null  object 
 6   Age              10000 non-null  int64  
 7   Tenure           10000 non-null  int64  
 8   Balance          10000 non-null  float64
 9   NumOfProducts    10000 non-null  int64  
 10  HasCrCard        10000 non-null  int64  
 11  IsActiveMember   10000 non-null  int64  
 12  EstimatedSalary  10000 non-null  float64
 13  Exited           10000 non-null  int64  
dtypes: float64(2), int64(9), object(3)
memory usage: 1.1+ MB


In [8]:
df = df.drop(columns={'RowNumber','CustomerId', 'Surname'} , axis=1 )

In [9]:
df.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 10000 entries, 0 to 9999
Data columns (total 11 columns):
 #   Column           Non-Null Count  Dtype  
---  ------           --------------  -----  
 0   CreditScore      10000 non-null  int64  
 1   Geography        10000 non-null  object 
 2   Gender           10000 non-null  object 
 3   Age              10000 non-null  int64  
 4   Tenure           10000 non-null  int64  
 5   Balance          10000 non-null  float64
 6   NumOfProducts    10000 non-null  int64  
 7   HasCrCard        10000 non-null  int64  
 8   IsActiveMember   10000 non-null  int64  
 9   EstimatedSalary  10000 non-null  float64
 10  Exited           10000 non-null  int64  
dtypes: float64(2), int64(7), object(2)
memory usage: 859.5+ KB


In [10]:
df = pd.get_dummies(df, drop_first=True)

<div class="alert alert-info">
    <b>Комментарий студента:</b> Применим технику прямого кодирования, чтобы преобразовать категориальные переменные в количественные. Для каждого замененного категориального столбца в таблице появятся новые количественные столбцы. Поскольку они сильно связаны между собой, и чтобы не попасть в дамми-ловушку, я указал аргумент drop_first=True
</div>

In [11]:
scaler = StandardScaler() # разделю выборки
target = df['Exited']
features = df.drop('Exited', axis=1)
features_train, features_2, target_train, target_2 = train_test_split(
    features, target, test_size=0.4, random_state=12345)
features_valid, features_test, target_valid, target_test = train_test_split(
    features_2, target_2, test_size=0.5, random_state=12345)

In [12]:
print(features_train.shape[0]) # проверю результат деления
print(features_valid.shape[0])
print(features_test.shape[0])

print(target_train.shape[0])
print(target_valid.shape[0])
print(target_test.shape[0])

6000
2000
2000
6000
2000
2000


In [13]:
scaler.fit(features_train) # преобразую обучающие, валидационные и тестовые выборки функцией transform()
features_train = scaler.transform(features_train)
features_valid = scaler.transform(features_valid)
features_test = scaler.transform(features_test)

<div class="alert alert-info">
    <b>Комментарий студента:</b> В этой части проекта я заполнил пропуски в столбце 'Tenure', перевёл столбцы с категориальными переменными в количественные, разделил данные на обучающую, валидационную и тестовую выборки, произвел масштабирование признаков с помощью метода стандартизации данных.
</div>

## Исследование задачи

<div class="alert alert-info">
    <b>Комментарий студента:</b> Посмотрим, как соотносятся значения целевого признака друг к другу. Для этого вызовем метод value_counts для обучающей выборки. Чтобы сразу увидеть соотношение величин, в параметрах укажем normalize=True :
</div>

In [14]:
target_train.value_counts(normalize=True)

0    0.800667
1    0.199333
Name: Exited, dtype: float64

<div class="alert alert-info">
    <b>Комментарий студента:</b> Как видим, классы несбалансированны, их соотношение примерно 4:1, а не 1:1. Баланс же классов наблюдается, если их количество примерно равно. Поэтому для определения точности модели метрика accuracy не подходит. Вместо будем использовать F1-меру. Также для сравнения будем сразу же подсчитывать метрику AUC-ROC. Обучим модель на обучающей выборке, и посчитаем значение F1-меры, AUC-ROC на данных валидационной выборки:
</div>

In [15]:
model = LogisticRegression(random_state=12345, solver='liblinear')
model.fit(features_train, target_train)
predictions = model.predict(features_valid)
valid_f1_score = f1_score(target_valid, predictions)
print("Значение F1-меры:", valid_f1_score)
probabilities_valid = model.predict_proba(features_valid)
probabilities_one_valid = probabilities_valid[:, 1]
auc_roc = roc_auc_score(target_valid, probabilities_one_valid)
print('Значение метрики AUC-ROC:', auc_roc)

SyntaxError: invalid syntax (1100065502.py, line 7)

<div class="alert alert-info">
    <b>Комментарий студента:</b> Настроим гиперпараметры нашего дерева решений. Самый важный гиперпараметр решающего дерева отвечает за глубину — max_depth. В цикле переберем все значения max_depth от 1 до 15 включительно. Выведем на экран значения max_depth, которые позволяют достичь максимального размера F1-меры и метрики AUC-ROC:
</div>

In [ ]:
col = ['max_depth', 'f1_score', 'auc_roc']
data = []

for depth in range(1, 16):
    model = DecisionTreeClassifier(random_state=12345, max_depth=depth)
    model.fit(features_train,target_train) 
    predictions = model.predict(features_valid)
    valid_f1_score = f1_score(target_valid, predictions)
    probabilities_valid = model.predict_proba(features_valid)
    probabilities_one_valid = probabilities_valid[:, 1]
    auc_roc = roc_auc_score(target_valid, probabilities_one_valid)
    data.append([depth, valid_f1_score, auc_roc])
table = pd.DataFrame(data = data, columns=col)
display(table[table['f1_score']==table['f1_score'].max()]) 
display(table[table['auc_roc']==table['auc_roc'].max()])

<div class="alert alert-info">
    <b>Комментарий студента:</b> Таким образом, значение F1-меры максимально и составляет 0.597 при значении max_depth=9. При этом максимальное значение метрики AUC-ROC (0.821) достигается при значении max_depth=5.
</div>

<div class="alert alert-info">
    <b>Комментарий студента:</b> Далее поработаем со случайным лесом. Чтобы управлять количеством деревьев в лесу, можно менять гиперпараметр n_estimators. Чем больше деревьев, тем дольше модель будет учиться, но результат станет лучше (и наоборот). Узнаем, какая величина n_estimators дает наилучший результат F1-меры и значения метрики AUC-ROC:
</div>

In [ ]:
col = ['n_estimators', 'f1_score', 'auc_roc']
data = []

for est in range(1, 50):
    model = RandomForestClassifier(random_state=12345, n_estimators=est) 
    model.fit(features_train,target_train) 
    predictions = model.predict(features_valid)
    f1 = f1_score(target_valid, predictions)
    probabilities_valid = model.predict_proba(features_valid)
    probabilities_one_valid = probabilities_valid[:, 1]
    auc_roc = roc_auc_score(target_valid, probabilities_one_valid)
    data.append([est, f1, auc_roc])
table = pd.DataFrame(data = data, columns=col)
display(table[table['f1_score']==table['f1_score'].max()]) 
display(table[table['auc_roc']==table['auc_roc'].max()])

<div class="alert alert-info">
    <b>Комментарий студента:</b> Наибольшее значение F1-меры в 0.5 достигается при величине гиперпараметра n_estimators равного 71. Максимальное значение метрики AUC-ROC (0.828) достигается при n_estimators=83.
</div>

<div class="alert alert-info">
    <b>Комментарий студента:</b> В этой части работы мы обнаружили, классы несбалансированны, их соотношение примерно 4:1. Также мы изучили модели для решения задачи классификации и подсчитали для каждой из них значение F1-меры и AUC-ROC, меняя (где есть возможность) значение гиперпараметров.
</div>

## Борьба с дисбалансом

In [ ]:
model = LogisticRegression(random_state=12345, solver='liblinear', class_weight='balanced')
model.fit(features_train, target_train)
predictions = model.predict(features_valid)
valid_f1_score = f1_score(target_valid, predictions)
print("Значение F1-меры:", valid_f1_score)
probabilities_valid = model.predict_proba(features_valid)
probabilities_one_valid = probabilities_valid[:, 1]
auc_roc = roc_auc_score(target_valid, probabilities_one_valid)
print('Значение метрики AUC-ROC:', auc_roc)

<div class="alert alert-info">
    <b>Комментарий студента:</b> Как видим значение F1-меры увеличилилось и стало 0.366 (было 0.293).
</div>

In [ ]:
col = ['max_depth', 'f1_score', 'auc_roc']
data = []

for depth in range(1, 16):
    model = DecisionTreeClassifier(random_state=12345, max_depth=depth, class_weight='balanced')
    model.fit(features_train,target_train) 
    predictions = model.predict(features_valid)
    valid_f1_score = f1_score(target_valid, predictions)
    probabilities_valid = model.predict_proba(features_valid)
    probabilities_one_valid = probabilities_valid[:, 1]
    auc_roc = roc_auc_score(target_valid, probabilities_one_valid)
    data.append([depth, valid_f1_score, auc_roc])
table = pd.DataFrame(data = data, columns=col)
display(table[table['f1_score']==table['f1_score'].max()]) 
display(table[table['auc_roc']==table['auc_roc'].max()])

<div class="alert alert-info">
    <b>Комментарий студента:</b> Как видим, теперь и максимальное значение F1-меры и значение метрики AUC-ROC достигаются при значении параметра max_depth = 5.
</div>

In [ ]:
col = ['n_estimators', 'f1_score', 'auc_roc']
data = []

for est in range(1, 100):
    model = RandomForestClassifier(random_state=12345, n_estimators=est, class_weight='balanced') 
    model.fit(features_train,target_train) 
    predictions = model.predict(features_valid)
    f1 = f1_score(target_valid, predictions)
    probabilities_valid = model.predict_proba(features_valid)
    probabilities_one_valid = probabilities_valid[:, 1]
    auc_roc = roc_auc_score(target_valid, probabilities_one_valid)
    data.append([est, f1, auc_roc])
table = pd.DataFrame(data = data, columns=col)
display(table[table['f1_score']==table['f1_score'].max()]) 
display(table[table['auc_roc']==table['auc_roc'].max()])

<div class="alert alert-info">
    <b>Комментарий студента:</b> Как видим, взвешивание классов совершенно изменило результаты. Теперь лучши показатели метрик достигаются при других величинах параметра n_estimators(количество деревьев в лесу):

максимальное значение F1-меры: 0.485, при n_estimators = 8
максимальное значение метрики AUC-ROC: 0.837, при n_estimators = 98
</div>

<div class="alert alert-info">
    <b>Комментарий студента:</b> Далее рассмотрим такой вид борьбы с дисбалансом, как изменение порога классификации. Посмотрим, как меняются показатели F1-меры и метрики AUC-ROC с изменением значения порога от 0 до 1 для модели логистической регрессии:
</div>

In [ ]:
model = LogisticRegression(random_state=12345, solver='liblinear')
model.fit(features_train, target_train)
probabilities_valid = model.predict_proba(features_valid)
probabilities_one_valid = probabilities_valid[:, 1]

col = ['порог', 'f1_score', 'auc_roc']
data = []
for threshold in np.arange(0, 1, 0.02):
    predicted_valid = probabilities_one_valid > threshold
    valid_f1_score = f1_score(target_valid, predicted_valid )
    auc_roc = roc_auc_score(target_valid, probabilities_one_valid)
    data.append([threshold, valid_f1_score, auc_roc])
table = pd.DataFrame(data = data, columns=col)
display(table[table['f1_score']==table['f1_score'].max()]) 
display(table[table['auc_roc']==table['auc_roc'].max()].head())

<div class="alert alert-info">
    <b>Комментарий студента:</b> Максимально значение F1-меры в 0.370 достишается при порог = 0.14. Метрика AUC-ROC равна 0.611.
</div>

In [ ]:
model =  DecisionTreeClassifier(random_state=12345, max_depth=5, class_weight='balanced')
model.fit(features_train, target_train)
probabilities_valid = model.predict_proba(features_valid)
probabilities_one_valid = probabilities_valid[:, 1]

col = ['порог', 'f1_score', 'auc_roc']
data = []
for threshold in np.arange(0, 1, 0.02):
    predicted_valid = probabilities_one_valid > threshold
    valid_f1_score = f1_score(target_valid, predicted_valid )
    auc_roc = roc_auc_score(target_valid, probabilities_one_valid)
    data.append([threshold, valid_f1_score, auc_roc])
table = pd.DataFrame(data = data, columns=col)
display(table[table['f1_score']==table['f1_score'].max()]) 
display(table[table['auc_roc']==table['auc_roc'].max()].head())

<div class="alert alert-info">
    <b>Комментарий студента:</b> Как видим, максимальное значение F1-меры (0.608) достигается при нескольких величинах порога. Примем за оптимальную наиболее близкую к величине по умолчанию (0.5) - 0.58. Значение метрики AUC-ROC при этом также максимальное и составляет 0.827.
</div>

In [ ]:
model = RandomForestClassifier(random_state=12345, n_estimators=9, class_weight='balanced') 
model.fit(features_train, target_train)
probabilities_valid = model.predict_proba(features_valid)
probabilities_one_valid = probabilities_valid[:, 1]

col = ['порог', 'f1_score', 'auc_roc']
data = []
for threshold in np.arange(0, 1, 0.02):
    predicted_valid = probabilities_one_valid > threshold
    valid_f1_score = f1_score(target_valid, predicted_valid )
    auc_roc = roc_auc_score(target_valid, probabilities_one_valid)
    data.append([threshold, valid_f1_score, auc_roc])
table = pd.DataFrame(data = data, columns=col)
display(table[table['f1_score']==table['f1_score'].max()]) 
display(table[table['auc_roc']==table['auc_roc'].max()].head())

<div class="alert alert-info">
    <b>Комментарий студента:</b> Как видим, максимальное показатели достигается при нескольких величинах порога.
</div>

In [ ]:
best_model = None
best_accuracy = 0
best_est = 0
best_depth = 0
best_f1_score = 0

for est in range(10, 51, 5):
    for depth in range (1, 21):
        model = RandomForestClassifier(random_state=12345, n_estimators=est, max_depth=depth, class_weight='balanced') 
        model.fit(features_train, target_train)
        accuracy = model.score(features_valid, target_valid) 
        predicted_valid = model.predict(features_valid)
        f1 = f1_score(target_valid, predicted_valid)
        if f1 > best_f1_score:
            best_model = model
            best_accuracy = accuracy
            best_est = est
            best_depth = depth
            best_f1_score = f1
print("Accuracy по наилучшей f1_score модели на валидационной выборке:", best_accuracy)
print("Количество деревьев:", best_est)
print("Оптимальная глубина дерева:", best_depth)
print("Лучшая F1_score:", best_f1_score)

## Тестирование модели

In [ ]:
model =  DecisionTreeClassifier(random_state=12345, max_depth=5, class_weight='balanced')
model.fit(features_train, target_train)
probabilities_valid = model.predict_proba(features_valid)
probabilities_one_valid = probabilities_valid[:, 1]

threshold = 0.58
predicted_valid = probabilities_one_valid > threshold
valid_f1_score = f1_score(target_valid, predicted_valid )
auc_roc = roc_auc_score(target_valid, probabilities_one_valid)
print('Валидационная выборка:')
print('F1-мера:', valid_f1_score)
print('AUC-ROC:',auc_roc)
probabilities_test = model.predict_proba(features_test)
probabilities_one_test = probabilities_test[:, 1]
predicted_test = probabilities_one_test > threshold
test_f1_score = f1_score(target_test, predicted_test )
auc_roc_test = roc_auc_score(target_test, probabilities_one_test)
print('Тестовая выборка:')
print('F1-мера:',test_f1_score)
print('AUC-ROC:',auc_roc_test)


In [ ]:
names = ['stratified', 'most_frequent', 'prior', 'uniform']
data = []
col = ['name', 'f1_score_valid', 'auc_roc_valid', 'f1_score_test', 'auc_roc_test']
for name in names:
    new_dummy_classifier = DummyClassifier(strategy=name)
    new_dummy_classifier.fit(features_train, target_train)

    predictions_valid = new_dummy_classifier.predict(features_valid)
    valid_accuracy = accuracy_score(target_valid, predictions)
    valid_f1_score = f1_score(target_valid, predictions_valid )
    auc_roc_valid = roc_auc_score(target_valid, predictions_valid)
    test_predictions = new_dummy_classifier.predict(features_test)
    test_accuracy = accuracy_score(target_test, test_predictions)
    test_f1_score = f1_score(target_test, test_predictions )
    auc_roc_test = roc_auc_score(target_test, test_predictions)
    data.append([name, valid_f1_score, auc_roc_valid, test_f1_score, auc_roc_test])
table = pd.DataFrame(data = data, columns=col)
display(table[table['f1_score_test']==table['f1_score_test'].max()]) 
display(table[table['f1_score_valid']==table['f1_score_valid'].max()].head())


<div class="alert alert-info">
    <b>Комментарий студента:</b> Как видим, показатели у лучшего (при значении параметра strategy='uniform') из фиктивных классификаторов намного ниже, чем у нашей модели, и находятся на уровне 0.29 для F1-меры, а также 0.498 - для метрики AUC-ROC.
</div>

<div class="alert alert-info">
    <b>Комментарий студента v2:</b> 
    В качестве лучшей модели была выбрана модель решающего дерева при значении гиперпараметра max_depth = 5, указании атрибута class_weight='balanced' f1, равным 0.597.
 
- На валидационной выборке модель следующие результаты:
* F1-мера: 0.608
* AUC-ROC: 0.831
- На тестовой выборке модель следующие результаты:

* F1-мера: 0.60
* AUC-ROC: 0.836
Проверку на адекватность модель выдержала.
</div>

<div class="alert alert-info">
    <b>Комментарий студента:</b> ОБЩИЙ ВЫВОД: В качестве лучшей модели была выбрана модель решающего дерева при значении гиперпараметра max_depth = 5, указании атрибута class_weight='balanced' f1, равным 0.597.
</div>